In [1]:
import requests
import folium
import pandas as pd

# 1. 获取实时数据
status_url = "https://api.cyclocity.fr/contracts/dublin/gbfs/station_status.json"
info_url = "https://api.cyclocity.fr/contracts/dublin/gbfs/station_information.json"

# 获取站点位置信息和实时状态
info_data = requests.get(info_url).json()['data']['stations']
status_data = requests.get(status_url).json()['data']['stations']

# 转为 DataFrame 方便合并
df_info = pd.DataFrame(info_data)
df_status = pd.DataFrame(status_data)
df = pd.merge(df_info, df_status, on='station_id')

# 2. 初始化地图 (以都柏林市中心为中心)
m = folium.Map(location=[53.3498, -6.2603], zoom_start=14, tiles='cartodbpositron')

# 3. 将站点添加到地图
for _, row in df.iterrows():
    # 定义颜色逻辑：没车红色，车少橙色，车多绿色
    available = row['num_bikes_available']
    color = 'green' if available > 5 else 'orange' if available > 0 else 'red'
    
    folium.CircleMarker(
        location=[row['lat'], row['lon']],
        radius=7,
        popup=f"<b>{row['name']}</b><br>Available: {available}/{row['capacity']}",
        color=color,
        fill=True,
        fill_opacity=0.7
    ).add_to(m)

# 4. 在 Jupyter 中直接显示地图
m